In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
import os
from shutil import copy2
import csv
import pickle
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_hub as hub
import pandas as pd
from PIL import Image
import cv2

In [2]:
pd.set_option("display.precision", 8)

In [3]:
data_root = ("../Datasets/")

In [4]:
IMAGE_SHAPE = (224, 224)
TRAINING_DATA_DIR = str(data_root)

datagen_kwargs = dict(rescale=1./255, validation_split=.20)
valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**datagen_kwargs)
valid_generator = valid_datagen.flow_from_directory(
    TRAINING_DATA_DIR, 
    subset="validation", 
    shuffle=True,
    target_size=IMAGE_SHAPE
)

train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**datagen_kwargs)
train_generator = train_datagen.flow_from_directory(
    TRAINING_DATA_DIR, 
    subset="training", 
    shuffle=True,
    target_size=IMAGE_SHAPE)

Found 7 images belonging to 16 classes.
Found 44 images belonging to 16 classes.


In [5]:
print (train_generator.class_indices)

{'Adrenaline': 0, 'Ethane': 1, 'Ethene': 2, 'Ethylene': 3, 'Isopentane': 4, 'M-xykene (1,3 - dimethylbenzene)': 5, 'Neopentane': 6, 'P-xylene (1,4 - dimethylbenzene)': 7, 'Phenylalanine': 8, 'Propylene': 9, 'acetaminophen': 10, 'acetylsalicylic': 11, 'benzene': 12, 'ibuprofen': 13, 'o-xylene (1,2 - dimethylbenzene)': 14, 'unknown': 15}


In [6]:
labels = '\n'.join(sorted(train_generator.class_indices.keys()))
with open('labels.txt', 'w') as f:
  f.write(labels)
!cat labels.txt

IMAGE_SIZE = 224
model = tf.keras.Sequential([
  hub.KerasLayer("https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4", 
                 output_shape=[1280],
                 trainable=False),
  tf.keras.layers.Dropout(0.4),
  tf.keras.layers.Dense(train_generator.num_classes, activation='softmax')
])
model.build([None, 224, 224, 3])

model.summary()

'cat' is not recognized as an internal or external command,
operable program or batch file.


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 1280)              2257984   
_________________________________________________________________
dropout (Dropout)            (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 16)                20496     
Total params: 2,278,480
Trainable params: 20,496
Non-trainable params: 2,257,984
_________________________________________________________________


In [7]:
optimizer = tf.keras.optimizers.Adam(lr=1e-3)
model.compile(
  optimizer=optimizer,
  loss='categorical_crossentropy',
  metrics=['acc'])
steps_per_epoch = np.ceil(train_generator.samples/train_generator.batch_size)
val_steps_per_epoch = np.ceil(valid_generator.samples/valid_generator.batch_size)

hist = model.fit(
    train_generator, 
    epochs=100,
    verbose=1,
    steps_per_epoch=steps_per_epoch,
    validation_data=valid_generator,
    validation_steps=val_steps_per_epoch).history

Epoch 1/100
2/2 [==============================] - 1s 389ms/step - loss: 2.7826 - acc: 0.1136 - val_loss: 1.1755 - val_acc: 0.8571
Epoch 2/100
2/2 [==============================] - 0s 125ms/step - loss: 1.8860 - acc: 0.4318 - val_loss: 0.6246 - val_acc: 0.8571
Epoch 3/100
2/2 [==============================] - 1s 359ms/step - loss: 1.7317 - acc: 0.5682 - val_loss: 0.3826 - val_acc: 0.8571
Epoch 4/100
2/2 [==============================] - 1s 359ms/step - loss: 1.5224 - acc: 0.5682 - val_loss: 0.2690 - val_acc: 0.8571
Epoch 5/100
2/2 [==============================] - 0s 130ms/step - loss: 1.1846 - acc: 0.6364 - val_loss: 0.2039 - val_acc: 0.8571
Epoch 6/100
2/2 [==============================] - 0s 133ms/step - loss: 1.1024 - acc: 0.5909 - val_loss: 0.1575 - val_acc: 1.0000
Epoch 7/100
2/2 [==============================] - 0s 130ms/step - loss: 1.0849 - acc: 0.6364 - val_loss: 0.1254 - val_acc: 1.0000
Epoch 8/100
2/2 [==============================] - 0s 129ms/step - loss: 0.7693 - a

2/2 [==============================] - 0s 123ms/step - loss: 0.0237 - acc: 1.0000 - val_loss: 0.0049 - val_acc: 1.0000
Epoch 64/100
2/2 [==============================] - 1s 335ms/step - loss: 0.0197 - acc: 1.0000 - val_loss: 0.0049 - val_acc: 1.0000
Epoch 65/100
2/2 [==============================] - 0s 124ms/step - loss: 0.0268 - acc: 1.0000 - val_loss: 0.0049 - val_acc: 1.0000
Epoch 66/100
2/2 [==============================] - 0s 124ms/step - loss: 0.0258 - acc: 1.0000 - val_loss: 0.0049 - val_acc: 1.0000
Epoch 67/100
2/2 [==============================] - 1s 368ms/step - loss: 0.0235 - acc: 1.0000 - val_loss: 0.0049 - val_acc: 1.0000
Epoch 68/100
2/2 [==============================] - 0s 122ms/step - loss: 0.0131 - acc: 1.0000 - val_loss: 0.0049 - val_acc: 1.0000
Epoch 69/100
2/2 [==============================] - 0s 121ms/step - loss: 0.0234 - acc: 1.0000 - val_loss: 0.0048 - val_acc: 1.0000
Epoch 70/100
2/2 [==============================] - 0s 120ms/step - loss: 0.0261 - acc: 1

In [8]:
img = cv2.imread("../Datasets/unknown/five.png")
res=cv2.resize(img ,dsize=(224,224), interpolation=cv2.INTER_CUBIC)
npimg=np.array(res)
print(npimg.shape)
sw=np.moveaxis(npimg,0,0)
rr=np.expand_dims(sw,0)
print(rr.shape)





val_image_batch, val_label_batch = next(iter(valid_generator))
true_label_ids = np.argmax(val_label_batch, axis=-1)

print("here")
#print(val_image_batch)
print(val_image_batch.shape)
dataset_labels = sorted(train_generator.class_indices.items(), key=lambda pair:pair[1])
dataset_labels = np.array([key.title() for key, value in dataset_labels])


tf_model_predictions = model.predict(rr)
print("Prediction results shape:", tf_model_predictions.shape)

predicted_ids = np.argmax(tf_model_predictions, axis=-1)
predicted_labels = dataset_labels[predicted_ids]
print(predicted_labels)


(224, 224, 3)
(1, 224, 224, 3)
here
(7, 224, 224, 3)
Prediction results shape: (1, 16)
['Benzene']


In [9]:
model.save('multi2.h5')